In [1]:
import socket,cv2,threading
import numpy as np

sock = socket.socket()
print(socket.gethostbyname(socket.gethostname()))
sock.bind((socket.gethostbyname(socket.gethostname()), 1234))
sock.listen()
session, address = sock.accept()
print(session.recv(4*1024)) 

cameraIndex = 'http://192.168.1.7:8080/video'
camera = cv2.VideoCapture(cameraIndex) 

def sender():
    while True:
        status, photo = camera.read()
        photo = cv2.resize(photo, (640, 480))
        if status:
            session.send(np.ndarray.tobytes(photo))
        else: print("Not in frame")

def receiver():
    print("Entered")
    while True:
        data = session.recv(10000000000)
        if(data == b'finished'):
            print("Finished")
            camera.release()
            session.close()
        else:
            photo =  np.frombuffer(data, dtype=np.uint8)
            if len(photo) == 640*480*3:
                cv2.imshow('From Client side', photo.reshape(480, 640, 3))
                if cv2.waitKey(80) == 13:
                    session.send(b'finished')
                    camera.release()
                    cv2.destroyAllWindows()
                    exit()
                
threading.Thread(target=sender).start()
threading.Thread(target=receiver).start()

192.168.56.1
b'\x90\x89\x90\x8e\x87\x8e\x8f\x89\x90\x91\x8b\x93\x8f\x89\x91\x8e\x87\x91\x92\x8c\x94\x97\x90\x98\x99\x91\x9b\x9d\x97\xa0\x9a\x96\x9e\x96\x94\x9c\x98\x96\x9f\x98\x99\xa0\x96\x98\x9f\x99\x99\xa2\x9d\x97\xa3\xa6\x98\xa7\xa9\x98\xa9\xac\x96\xa9\xae\x96\xaa\xb0\x96\xab\xad\x95\xab\xab\x95\xab\xa8\x96\xaa\xa1\x95\xa7\xa0\x99\xa8\x9d\x9a\xa6\x9a\x97\xa0\x9c\x9a\x9f\xa3\x9f\xa4\xa4\x9f\xa1\xa4\x9d\x9e\xa8\x9e\xa0\xaa\x9d\xa3\xac\x9c\xa7\xa7\x94\xa4\xaa\x94\xaa\xac\x93\xaa\xae\x92\xac\xb2\x95\xae\xb0\x93\xac\xae\x93\xaa\xaf\x96\xab\xb0\x97\xac\xaf\x95\xaa\xad\x93\xaa\xa8\x93\xa8\x9d\x90\xa2\x98\x91\xa0\x98\x90\xa1\x99\x92\xa3\x99\x93\xa2\x97\x93\xa1\x95\x93\xa0\x95\x93\x9f\x95\x92\x9d\x94\x8f\x9a\x94\x8e\x99\x97\x8e\x9a\x9b\x90\x9d\x9e\x90\x9f\xa0\x90\xa1\xa6\x91\xa3\xa8\x8f\xa3\xab\x8e\xa4\xa9\x8e\xa3\xa6\x8b\x9f\xa4\x8b\x9d\xa3\x8b\x9d\xa3\x8c\x9e\xa3\x8d\x9f\xa1\x8e\x9f\xa1\x90\xa1\xa0\x91\xa3\x9f\x92\xa5\x9f\x91\xa4\x9d\x8e\xa3\x9c\x8b\xa2\x9c\x89\xa0\x9b\x88\x9e\x9b\x88\x9c\